In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

from source_code.SqlCo import Sqldd
from source_code.backpip import data_up_pip

Autosave disabled
Local Sec Env loaded


In [2]:
## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today() - timedelta(days=1)}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today() - timedelta(days=1)}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
388,AFG,Asia,Afghanistan,2021-03-18,56044.0,28.0,18.143,2462.0,2.0,1.571,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.511
788,OWID_AFR,NaN,Africa,2021-03-18,4075556.0,11332.0,9806.143,108932.0,261.0,275.143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1176,ALB,Europe,Albania,2021-03-18,119528.0,590.0,583.714,2106.0,14.0,14.857,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.795


In [3]:
## dump the file for using it in other nb
# c.to_csv(f"data\\Base_Files\\full_data{date.today()}.csv", index=False)

In [4]:
## insert yesterday reporting to bdd
# try:
#     verif_data = verif_data.replace({np.nan: 'Null'})
#     data_job = data_up_pip(verif_data) 
#     data_job.up_baseline()
#     data_job.clean_leave()
# except:
#     print("BDD job Failed -- Normal if you not have acces to main code")

In [5]:
%%time
## Load data from days -1 to evaluate the performances of model
yesterday_date = date.today() - timedelta(days=2)
yesterday_date = str(yesterday_date)
this_date = date.today() - timedelta(days=1)
this_date = str(this_date)

tip = Sqldd()
cnx, cursor = tip.get_bdd_co()
data_load = pd.read_sql(f"SELECT * FROM cov_aipred WHERE date='{yesterday_date}';", con=cnx) 

Wall time: 1.88 s


In [8]:
def Eval(c,data_load,country):
    """ function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    df = df[df['date'].isin([f"{this_date}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]

    res_tempo = data_load[data_load['location'].isin([f"{country}"])]
    res_tempo= res_tempo.reset_index()
    rez1 = res_tempo.at[0,"total_cases_predict"]
    rez2 = res_tempo.at[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2
    
    ind = 0    
    follow_df = pd.DataFrame()
    follow_df.at[ind,"date"] = v0
    follow_df.at[ind,"location"] = country
    follow_df.at[ind,"total_cases_predict"] = rez1
    follow_df.at[ind,"total_cases_real"] = v1
    follow_df.at[ind,"total_deaths_predict"] = rez2
    follow_df.at[ind,"total_deaths_real"] = v2
    follow_df.at[ind,"error_abs_cases"] = errorCase1
    follow_df.at[ind,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [9]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
# follow_df4 = Eval(c,data_load,"Belgium")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [10]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,location,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2021-03-18,France,4192610.0,4241973.0,91720.0,91833.0,-49363.0,-113.0
0,2021-03-18,China,101457.0,101481.0,4841.0,4839.0,-24.0,2.0
0,2021-03-18,Italy,3295560.0,3306711.0,103679.0,103855.0,-11151.0,-176.0
0,2021-03-18,United States,29849700.0,29667305.0,541860.0,539698.0,182395.0,2162.0
0,2021-03-18,World,121926000.0,121788101.0,2694150.0,2691035.0,137899.0,3115.0
0,2021-03-18,United Kingdom,4317140.0,4294328.0,126603.0,126163.0,22812.0,440.0
0,2021-03-18,Germany,2623480.0,2639258.0,74567.0,74402.0,-15778.0,165.0
0,2021-03-18,Iran,1778320.0,1778645.0,61782.0,61581.0,-325.0,201.0
0,2021-03-18,Turkey,2948320.0,2950603.0,29865.0,29777.0,-2283.0,88.0
0,2021-03-18,Brazil,11670400.0,11780820.0,282917.0,287499.0,-110420.0,-4582.0


In [ ]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
# rapport.to_csv(f"data\\Rapport\\rap{date.today()}.csv")

In [11]:
# insert yesterday reporting to bdd
try:
    data_job = data_up_pip(rapport) 
    data_job.up_rapp()
    data_job.clean_leave()
except:
    print("BDD job Failed -- Normal if you not have acces to main code")

 Insert Rapport Done !! 
